In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df=pd.read_csv('thesis dataset copy - Sheet1 (2).csv')
df.head()

,Date,Chemical Name,Price/Kg,Packing,Purity,Chemical Nature,Supplier Country,Storing Container,Supplier Company,Boiling Point,Composition,Usage,Price/MT,Quality
0,3/1/2024,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
1,3/2/2024,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
2,3/3/2024,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
3,3/4/2024,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
4,3/5/2024,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good


In [ ]:
df=df.drop(columns=["Date"])
df.head()

,Chemical Name,Price/Kg,Packing,Purity,Chemical Nature,Supplier Country,Storing Container,Supplier Company,Boiling Point,Composition,Usage,Price/MT,Quality
0,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
1,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
2,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
3,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good
4,Titanium Dioxide,432,25,72,Powder,Korea,Paper Bag,Cosmo Chemicals Co. Ltd.,2500.0,"Titanum, Oxygen",printing and paste liquify,2870,Good


In [ ]:
columns_to_one_hot_encode=[
    'Chemical Name', 'Chemical Nature','Supplier Country','Storing Container','Supplier Company','Composition','Usage'
]

In [ ]:
one_hot_encoder = OneHotEncoder(drop='first', sparse=False)
encoded_columns = one_hot_encoder.fit_transform(df[columns_to_one_hot_encode])
encoded_df = pd.DataFrame(encoded_columns, columns=one_hot_encoder.get_feature_names_out(columns_to_one_hot_encode))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
df = df.drop(columns=columns_to_one_hot_encode)
df=pd.concat([df,encoded_df],axis=1)

In [ ]:
label_encoder = LabelEncoder()
df['Quality'] = label_encoder.fit_transform(df['Quality'])

In [ ]:
pd=pd.DataFrame(df)

In [ ]:
df.fillna(df.mean(), inplace=True)
X = df.drop(columns=['Quality'])
y = df['Quality']
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier




# Define parameter grid for XGBoost and LightGBM
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}

# Initialize classifiers
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
lgb_clf = LGBMClassifier()

# Perform Grid Search for XGBoost and LightGBM
grid_xgb = GridSearchCV(xgb_clf, param_grid, cv=5, scoring='accuracy')
grid_lgb = GridSearchCV(lgb_clf, param_grid, cv=5, scoring='accuracy')

# Fit the models
grid_xgb.fit(X_train, y_train)
grid_lgb.fit(X_train, y_train)

# Best parameters for XGBoost and LightGBM
print("Best parameters for XGBoost:", grid_xgb.best_params_)
print("Best parameters for LightGBM:", grid_lgb.best_params_)

# Parameter grid for AdaBoost with DecisionTree as base estimator
param_grid_ada = {
    'base_estimator__max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200]
}

# Initialize AdaBoost with DecisionTreeClassifier as base estimator
ada_clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

# Perform Grid Search for AdaBoost
grid_ada = GridSearchCV(ada_clf, param_grid_ada, cv=5, scoring='accuracy')

# Fit the model
grid_ada.fit(X_train, y_train)

# Best parameters for AdaBoost
print("Best parameters for AdaBoost:", grid_ada.best_params_)

# Predictions using the best estimators
xgb_best = grid_xgb.best_estimator_
ada_best = grid_ada.best_estimator_
lgb_best = grid_lgb.best_estimator_

y_pred_xgb = xgb_best.predict(X_test)
y_pred_ada = ada_best.predict(X_test)
y_pred_lgb = lgb_best.predict(X_test)

# Evaluation
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')  # Adjust 'average' as needed
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

print("Evaluation for XGBoost:")
acc_xgb, prec_xgb, rec_xgb, f1_xgb = evaluate_model(y_test, y_pred_xgb)
print(f"Accuracy: {acc_xgb}, Precision: {prec_xgb}, Recall: {rec_xgb}, F1 Score: {f1_xgb}")

print("Evaluation for AdaBoost:")
acc_ada, prec_ada, rec_ada, f1_ada = evaluate_model(y_test, y_pred_ada)
print(f"Accuracy: {acc_ada}, Precision: {prec_ada}, Recall: {rec_ada}, F1 Score: {f1_ada}")

print("Evaluation for LightGBM:")
acc_lgb, prec_lgb, rec_lgb, f1_lgb = evaluate_model(y_test, y_pred_lgb)
print(f"Accuracy: {acc_lgb}, Precision: {prec_lgb}, Recall: {rec_lgb}, F1 Score: {f1_lgb}")


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3809, number of negative: 2149
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000335 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 258
[LightGBM] [Info] Number of data points in the train set: 5958, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.639308 -> initscore=0.572364
[LightGBM] [Info] Start training from score 0.572364
[LightG

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Best parameters for AdaBoost: {'base_estimator__max_depth': 7, 'learning_rate': 0.01, 'n_estimators': 100}
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Evaluation for XGBoost:
Accuracy: 0.9323308270676691, Precision: 0.9080188679245284, Recall: 0.9922680412371134, F1 Score: 0.9482758620689655
Evaluation for AdaBoost:
Accuracy: 0.9339419978517723, Precision: 0.9050583657587549, Recall: 0.9991408934707904, F1 Score: 0.9497754185381789
Evaluation for LightGBM:
Accuracy: 0.935016111707841, Precision: 0.9155378486055777, Recall: 0.9871134020618557, F1 Score: 0.9499793303017776


for svm , logistic regression, random forest


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier



# Define parameter grids
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

param_grid_svm = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize classifiers
lr_clf = LogisticRegression(max_iter=10000)
svm_clf = SVC()
rf_clf = RandomForestClassifier()

# Perform Grid Search for each classifier
grid_lr = GridSearchCV(lr_clf, param_grid_lr, cv=5, scoring='accuracy')
grid_svm = GridSearchCV(svm_clf, param_grid_svm, cv=5, scoring='accuracy')
grid_rf = GridSearchCV(rf_clf, param_grid_rf, cv=5, scoring='accuracy')

# Fit the models
grid_lr.fit(X_train, y_train)
grid_svm.fit(X_train, y_train)
grid_rf.fit(X_train, y_train)

# Best parameters
print("Best parameters for Logistic Regression:", grid_lr.best_params_)
print("Best parameters for SVM:", grid_svm.best_params_)
print("Best parameters for Random Forest:", grid_rf.best_params_)

# Predictions using the best estimators
lr_best = grid_lr.best_estimator_
svm_best = grid_svm.best_estimator_
rf_best = grid_rf.best_estimator_

y_pred_lr = lr_best.predict(X_test)
y_pred_svm = svm_best.predict(X_test)
y_pred_rf = rf_best.predict(X_test)

# Evaluation
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')  # Adjust 'average' as needed
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

print("Evaluation for Logistic Regression:")
acc_lr, prec_lr, rec_lr, f1_lr = evaluate_model(y_test, y_pred_lr)
print(f"Accuracy: {acc_lr}, Precision: {prec_lr}, Recall: {rec_lr}, F1 Score: {f1_lr}")

print("Evaluation for SVM:")
acc_svm, prec_svm, rec_svm, f1_svm = evaluate_model(y_test, y_pred_svm)
print(f"Accuracy: {acc_svm}, Precision: {prec_svm}, Recall: {rec_svm}, F1 Score: {f1_svm}")

print("Evaluation for Random Forest:")
acc_rf, prec_rf, rec_rf, f1_rf = evaluate_model(y_test, y_pred_rf)
print(f"Accuracy: {acc_rf}, Precision: {prec_rf}, Recall: {rec_rf}, F1 Score: {f1_rf}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour

Best parameters for Logistic Regression: {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'}
Best parameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'poly'}
Best parameters for Random Forest: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 100}
Evaluation for Logistic Regression:
Accuracy: 0.7728249194414608, Precision: 0.7505070993914807, Recall: 0.9536082474226805, F1 Score: 0.8399545970488081
Evaluation for SVM:
Accuracy: 0.7916219119226638, Precision: 0.7628726287262872, Recall: 0.9673539518900344, F1 Score: 0.8530303030303029
Evaluation for Random Forest:
Accuracy: 0.9301825993555317, Precision: 0.9116242038216561, Recall: 0.9836769759450171, F1 Score: 0.9462809917355371
